### Árboles de decisión

Tenemos un CSV con distintos datos de hojas de árboles, con una estructura como esta

    id,species,margin1,shape1,texture1
    1,Acer_Opalus,0.007812,0.00064671,0.049805
    2,Pterocarya_Stenoptera,0.005859,0.00074942,0
    3,Quercus_Hartwissiana,0.005859,0.00097311,0.003906
    5,Tilia_Tomentosa,0,0.00045312,0.023438
    6,Quercus_Variabilis,0.005859,0.00068161,0.039062
    8,Magnolia_Salicifolia,0.070312,0.00039032,0.026367
    10,Quercus_Canariensis,0.021484,0.00092122,0.003906
    11,Quercus_Rubra,0,0.00077608,0.018555


In [1]:
import csv

def read_csv(fname):
    '''Definimos una forma de leer el CSV, de ahí tendrémos que extraer dos listas.
    
    Los inputs, características que definien a cada hoja.
    Los outputs, la clasificación que se le quiere dar.
    '''
    inputs = []
    outputs = []

    # Iteramos sobre todos las filas del archivo, la primera define los "headers"
    # y del resto, la propiedad [1] es su especie (la clasificación que buscamos),
    # los elementos a partir de ahí son sus caracterítisticas
    with open(fname, newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        headers = next(csvreader, None) # Skip the headers
        features = headers[2:]
        for row in csvreader:
            outputs.append(row[1])
            inputs.append(row[2:])

    # Para ayudar más adelante a la visualización, convertiremos los outputs (un string con la especie)
    # en un número (diferente para cada especie) y una lista que los relaciona con este número
    outputs, labels = transform_labels(outputs)
    return inputs, outputs, features, labels


def transform_labels(outputs):
    '''Método auxiliar, asignamos un número a cada etiqueta y construimos una lista con su correspondencia.'''

    label_set = list(set(outputs)) # Construimos una lista sin especies repetidas
    t_outputs = []
    for out in outputs: # Por cada elemento
        t_out = label_set.index(out) # Construimos una lista, donde su valor sea el índice que tiene
        t_outputs.append(t_out)      # en la lista de especies no-repetidas

    return t_outputs, label_set


inputs, outputs, features, labels = read_csv('data/train.csv')

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs)

print("OK")

OK


Una vez tenemos los datos, habrá que construir un árbol que realize la clasificación. Para esto utilizaremos la librería scikit-learn.

In [ ]:
import sklearn.tree # Importamos la librería
tree = sklearn.tree.DecisionTreeClassifier() # Decidimos que tipo de algoritmo queremos utilizar

# Y simplemente lo alimentamos con los datos que hemos extraído
tree.fit(X_train, y_train)

In [ ]:
# Ya podemos visualizar el árbol que se ha generado
sklearn.tree.export_graphviz(
    tree, out_file='data/tree.dot',
    feature_names=features,
    class_names=labels,
    label='none',
    filled=True,
)


import subprocess
# Hacemos una pequeña limpieza del grafico...
subprocess.call(['sed', '-r', r's/\[([0-9]|,| |\\n)+\]//g',  '-i', 'data/tree.dot'])
subprocess.call(['sed', '-r', r's/\\n([0-9.])+//g',  '-i', 'data/tree.dot'])
subprocess.call(['sed', '-r', r's/(\\n)+/\\n/g',  '-i', 'data/tree.dot'])
subprocess.call(['sed', '-r', r's/"0.0\\n/"/g',  '-i', 'data/tree.dot'])
subprocess.call(['dot', '-Tpng', 'data/tree.dot', '-o' 'data/tree.png'])
print("OK")

In [ ]:
from IPython.core.display import Image
Image('data/tree.png')

In [ ]:
# A partir de aquí podemos ya podemos probar los resultados del algoritmo
tree.score(X_test, y_test)